In [3]:
#Wo geschehen die meisten tödlichen Strassenunfälle?
#Einige Analysen
import csv
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

In [4]:
df = pd.read_csv("RoadTrafficAccidentLocations.csv")


In [5]:
df.head(3)

,AccidentUID,AccidentType,AccidentType_de,AccidentType_fr,AccidentType_it,AccidentType_en,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentSeverityCategory_fr,AccidentSeverityCategory_it,...,AccidentMonth_fr,AccidentMonth_it,AccidentMonth_en,AccidentWeekDay,AccidentWeekDay_de,AccidentWeekDay_fr,AccidentWeekDay_it,AccidentWeekDay_en,AccidentHour,AccidentHour_text
0,9FD6441F802C20A6E0430A865E3320A6,at0,Schleuder- oder Selbstunfall,dérapage ou perte de maîtrise,Incidente di sbandamento o per colpa propria,Accident with skidding or self-accident,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,1.0,01h-02h
1,99AE7B25356510B0E0430A865E3310B0,at4,Einbiegeunfall,accident en s'engageant sur une route,Incidente nell’immettersi in una strada,Accident when turning-into main road,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,1.0,01h-02h
2,9A096471BA579094E0430A865E339094,at00,Andere,autres,Altri,Other,as2,Unfall mit Schwerverletzten,accident avec blessés graves,Incidente con feriti gravi,...,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,2.0,02h-03h


In [6]:
#In diesem Jahr beginnt die Aufzeichnung
df['AccidentYear'].min()

2011

In [18]:
df.columns

Index(['AccidentUID', 'AccidentType', 'AccidentType_de', 'AccidentType_fr',
       'AccidentType_it', 'AccidentType_en', 'AccidentSeverityCategory',
       'AccidentSeverityCategory_de', 'AccidentSeverityCategory_fr',
       'AccidentSeverityCategory_it', 'AccidentSeverityCategory_en',
       'AccidentInvolvingPedestrian', 'AccidentInvolvingBicycle',
       'AccidentInvolvingMotorcycle', 'RoadType', 'RoadType_de', 'RoadType_fr',
       'RoadType_it', 'RoadType_en', 'AccidentLocation_CHLV95_E',
       'AccidentLocation_CHLV95_N', 'CantonCode', 'MunicipalityCode',
       'AccidentYear', 'AccidentMonth', 'AccidentMonth_de', 'AccidentMonth_fr',
       'AccidentMonth_it', 'AccidentMonth_en', 'AccidentWeekDay',
       'AccidentWeekDay_de', 'AccidentWeekDay_fr', 'AccidentWeekDay_it',
       'AccidentWeekDay_en', 'AccidentHour', 'AccidentHour_text'],
      dtype='object')

In [ ]:
df = df.drop(columns=['AccidentType_fr',
       'AccidentType_it', 'AccidentType_en', 'AccidentSeverityCategory_fr',
       'AccidentSeverityCategory_it', 'AccidentSeverityCategory_en', 'RoadType_fr',
       'RoadType_it', 'RoadType_en', 'AccidentMonth_fr',
       'AccidentMonth_it', 'AccidentMonth_en', 'AccidentWeekDay_fr', 'AccidentWeekDay_it',
       'AccidentWeekDay_en'])

In [21]:
#in diesem Jahr endet die Aufzeichnung
df['AccidentYear'].max()

2020

In [22]:
#Zahl der Einträge weil jeder Eintrag mit Jahr angegeben ist
df['AccidentYear'].count()

178217

In [23]:
#Ist es chronologisch geordnet? Ja. 
df.AccidentYear

0         2011
1         2011
2         2011
3         2011
4         2011
          ... 
178212    2020
178213    2020
178214    2020
178215    2020
178216    2020
Name: AccidentYear, Length: 178217, dtype: int64

In [24]:
#Zahl der Strassenunfälle pro Jahr. Erfreulich: Noch nie waren es so wenige wie 2020. Evtl wegen Corona?
df['AccidentYear'].value_counts()

2011    18990
2012    18148
2018    18033
2014    17803
2017    17799
2019    17761
2015    17736
2016    17577
2013    17473
2020    16897
Name: AccidentYear, dtype: int64

In [14]:
#Welche Art von Unfall gab es wie oft? 
#Legende: 
#at0 Schleuder- oder Selbstunfall 
#at1 Überholunfall oder Fahrstreifenwechsel 
#at2 Auffahrunfall 
#at3 Abbiegeunfall 
#at4 Einbiegeunfall 
#at5 Überqueren der Fahrbahn 
#at6 Frontalkollision 
#at7 Parkierunfall 
#at8 Fussgängerunfall 
#at9 Tierunfall 
#at00 Andere

UnfallTyp = df["AccidentType"].value_counts().sort_values(ascending=False)
UnfallTyp

at0     55801
at2     36924
at4     21682
at8     20580
at3     14443
at5      8983
at1      8330
at6      6938
at7      2268
at00     1440
at9       828
Name: AccidentType, dtype: int64

In [12]:
#Schleuder- oder Selbstunfälle gab es meisten, Unfälle mit Tiere am seltensten. Frage an mich: Unterschied zwischen Abbiege- und Einbiegeunfall? 

In [13]:
#Wie schwer waren die Unfälle?
#Legende
#as1 Unfall mit Getöteten
#as2 Unfall mit Schwerverletzten
#as3 Unfall mit Leichtverletzten
df['AccidentSeverityCategory'].value_counts()

as3    138868
as2     36954
as1      2395
Name: AccidentSeverityCategory, dtype: int64

In [14]:
#Die meisten Unfälle gingen glücklicherweise glimpflich aus. Aber wo sind die Unfälle mit Sachschaden? 

In [15]:
#Auf welchen Strassen geschahen die Unfälle?
df['RoadType'].value_counts()
#rt430 Autobahn
#rt431 Autostrasse
#rt432 Hauptstrasse
#rt433 Nebenstrasse
#rt434 Autobahnnebenanlage
#rt439 andere

rt432    89517
rt433    64591
rt430    16479
rt439     5924
rt431     1485
rt434      221
Name: RoadType, dtype: int64

In [16]:
#Für mich überraschend: Auf der Autobahn gibt es deutlich weniger Unfälle als auf Haupt- und Nebenstrassen. Fahren Menschen vorsichtiger wenn sie schneller fahren? Evtl mit Verkehrsverkommen quervergleichen. 



In [17]:
#In welchem Kanton gibt es am meisten Unfälle?
df['CantonCode'].value_counts()

ZH    32178
BE    25435
VD    16690
AG    11771
GE    11131
SG    10045
LU     9788
TI     8491
SO     6219
TG     5981
FR     5472
VS     5196
GR     4876
BL     4805
BS     3529
NE     2962
SZ     2952
ZG     2677
SH     1650
JU     1430
UR     1088
AR      964
GL      922
NW      868
OW      792
AI      305
Name: CantonCode, dtype: int64

In [18]:
#Scheint auf den ersten Blick mit Einwohnerzahl vergleichbar, was Sinn macht. Evtl genauer anschauen. 

In [19]:
#Gibt es einen Monat, in dem besonders viele Unfälle passieren?
df['AccidentMonth'].value_counts()

6     18903
8     17753
9     17672
7     17160
5     16603
10    15461
11    14114
4     13911
3     12655
12    12249
1     11375
2     10361
Name: AccidentMonth, dtype: int64

In [20]:
#Finde ich überraschend. Die meisten Unfälle passieren von Mai bis September. Nicht im Winter. Warum ist das so? Touristen? Kann man gut was machen "Achtung, jetzt beginnt die Unfallsaison", aber zuerst Gründe finden. 

In [21]:
#Und wie sieht es bei den Wochentagen aus?
#Legende
#aw401 Montag 
#aw402 Dienstag 
#aw403 Mittwoch 
#aw404 Donnerstag 
#aw405 Freitag 
#aw406 Samstag 
#aw407 Sonntag
df['AccidentWeekDay'].value_counts()

aw405    28986
aw404    27397
aw403    27360
aw402    26790
aw401    25521
aw406    24190
aw407    17973
Name: AccidentWeekDay, dtype: int64

In [22]:
#Die Sonntagsfahrer scheinen jedenfall sicherer zuf ahren als man annimmt. Warum am Freitag am meisten? Weil Leute schnell ins Weekend wollen? Oder nehmen mehr Leute das Auto um schneller zu Hause zu sein? 

In [23]:
df['AccidentHour'].value_counts()

17.0    19264
16.0    15432
18.0    13201
15.0    12605
14.0    11361
13.0    11263
7.0     10710
12.0    10460
11.0    10289
10.0     7996
8.0      7962
19.0     7932
9.0      7063
6.0      7007
20.0     4882
21.0     3871
22.0     3514
23.0     2780
5.0      2291
0.0      2162
1.0      1846
2.0      1515
4.0      1424
3.0      1382
Name: AccidentHour, dtype: int64

#Am frühen Abend/Nachmittag gibt es die meisten Unfälle. Evtl weil Menschen müde sind vom Arbeitstag?

In [24]:
#Aus diesen Daten lassen sich wohl einige spannende Dinge herauslesen. Zuerst interessiert mich, wie viele Unfälle es mit Schwerverletzten in welchem Jahr gab
df[df['AccidentSeverityCategory'] == 'as2']['AccidentYear'].value_counts()

2011    4110
2012    3867
2013    3859
2014    3818
2018    3640
2020    3619
2015    3612
2016    3548
2019    3454
2017    3427
Name: AccidentYear, dtype: int64

In [25]:
#Und mit Getöteten?
df[df['AccidentSeverityCategory'] == 'as1']['AccidentYear'].value_counts()

2011    312
2012    301
2013    257
2015    238
2014    229
2018    228
2020    224
2017    219
2016    208
2019    179
Name: AccidentYear, dtype: int64

In [26]:
#Die schweren Unfälle scheinen also ebenfalls zurückzugehen. Für meine Auswertung will ich mich zuerst aufs Jahr 2020 beziehen. 

In [27]:
#Nun möchte ich schauen, wie viele der Unfälle mit Getöteten welchem Unfalltypen entsprechen
df[df['AccidentSeverityCategory'] == 'as1']['AccidentType'].value_counts()

at0     1037
at8      506
at6      244
at1      141
at3      134
at2      100
at4       87
at5       80
at00      47
at7       14
at9        5
Name: AccidentType, dtype: int64

In [28]:
#Am meisten Menschen sterben also bei einem Schleuder- oder Selbstunfall. Danach folgen Unfälle mit Fussgängern und Auffahrunfälle. Diese Unfälle mit Fussgängern dünken mich spannend, da will ich mehr wissen. 

In [29]:
df[df['AccidentType'] == 'at8']['AccidentSeverityCategory'].value_counts()

as3    14358
as2     5716
as1      506
Name: AccidentSeverityCategory, dtype: int64

In [30]:
#In den vergangenen 10 Jahren kam es also zu über 6000 Verkehrsunfällen, bei denen Fussgänger schwer verletzt oder getötet wurden. Wie viele waren es 2019?

In [31]:
#Mich interessiert nun speziell das aktuellste Jahr. Dass es weniger Unfälle gab als in den Vorjahren weiss ich bereits, aber die möchte ich im Detail anschauen. Ich exkludiere darum das Jahr 2020.
df1 = df[df['AccidentYear'] == 2020]


In [32]:
#Fussgängerunfälle 2020
df1[df1['AccidentType'] == 'at8']['AccidentSeverityCategory'].value_counts()   

as3    1182
as2     420
as1      38
Name: AccidentSeverityCategory, dtype: int64

In [25]:
#Unfall mit Velobeteiligung – davon gab es über 40'000
Velounfall = df["AccidentInvolvingBicycle"].value_counts().sort_values(ascending=False)
Velounfall

False    137214
True      41003
Name: AccidentInvolvingBicycle, dtype: int64

In [26]:
#Unfall mit Töffbeteiligung
unfallmittoeff = df["AccidentInvolvingMotorcycle"].value_counts().sort_values(ascending=False)
unfallmittoeff

False    141038
True      37179
Name: AccidentInvolvingMotorcycle, dtype: int64

In [ ]:
#Knapp weniger als Fahrräder. Gibt aber wohl im Verhältnis deutlich mehr Fahrräder als Motorräder, darum Velos eher sicherer?

In [27]:
#Unfälle nach Strassentypen – am meist
Strassentyp = df["RoadType_de"].value_counts().sort_values(ascending=False)
Strassentyp

Hauptstrasse    89517
Nebenstrasse    64591
Autobahn        16479
andere           5924
Autostrasse      1485
Nebenanlage       221
Name: RoadType_de, dtype: int64

In [ ]:
#Auf Hauptstrassen gibt es die meisten Unfälle. Spannend: Nebenstrassen sind viel gefährlicher als Autobahnen. Sind Autostrassen so sicher weil es davon am wenigsten gibt?